In [2]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import text
import os

# ================== YOUR AIVEN CONNECTION ==================
host = "anly-615-project-anlyproject.g.aivencloud.com"
port = 23263
user = "avnadmin"
password = "AVNS_uZtAlXsQZVgdnkwXesP"
database = "defaultdb"

connection_string = f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}"
engine = create_engine(connection_string, connect_args={"ssl": {"ssl_mode": "REQUIRED"}}, echo=False)

print("Connected to Aiven MySQL successfully!")


Connected to Aiven MySQL successfully!


In [3]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT NOW() AS `current_time`"))
    print("Connected successfully!")
    print("Aiven server time:", result.fetchone()[0])

Connected successfully!
Aiven server time: 2025-11-18 19:59:23


In [17]:
with engine.connect() as conn:
    conn.execute(text("DROP TABLE IF EXISTS students100"))
    conn.execute(text("""
        CREATE TABLE students100 (
            id INT AUTO_INCREMENT PRIMARY KEY,
            last_name VARCHAR(50),
            first_name VARCHAR(50),
            ssn CHAR(11),
            gpa DECIMAL(3,2)
        ) ENGINE=InnoDB
    """))
    conn.commit()


In [13]:
students_df = pd.read_sql("SELECT * FROM students100", engine)


print(students_df)

Empty DataFrame
Columns: [id, last_name, first_name, ssn, gpa]
Index: []


In [14]:
df = pd.read_csv('students100.csv', header=None, names=['last_name', 'first_name', 'ssn', 'gpa'])

df.to_sql(
    name="students100",
    con=engine,
    if_exists="append",   # append into the table we just made
    index=False,
    method="multi",       # fast bulk insert
    chunksize=10000
)

print(f"Loaded {len(df)} rows into students100")

# Verify
pd.read_sql("SELECT * FROM students100 LIMIT 10", engine)

Loaded 100 rows into students100


,id,last_name,first_name,ssn,gpa
0,1,Aguirre,Margaret,198-00-1324,3.68
1,2,Sanders,Eric,559-00-2664,3.82
2,3,Cook,Stephanie,850-00-6327,3.83
3,4,Juarez,Michael,404-00-9769,3.59
4,5,Ford,George,567-00-9885,3.63
5,6,Turner,Robin,426-00-3925,3.70
6,7,Vargas,Randy,181-00-6592,3.23
7,8,Smith,Shirley,747-00-7204,3.49
8,9,Garcia,Patrick,927-00-3133,3.73
9,10,Mccoy,Kenneth,589-00-4991,3.62


In [ ]:
Nurtas  CODE